In [12]:
import os, sys
import logging
from HumanObj_videos_ResNet.lib.config import ConfigContext, parse_args, args
from HumanObj_videos_ResNet.predict.image import Image_processor

from HumanObj_videos_ResNet.lib.models import build_model

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd

In [2]:
ConfigContext.parsed_args = parse_args(["--configs_yml=configs/image.yml",
                                        '--inputs=demo/images', 
                                        '--output_dir=demo/image_results2', 
                                        '--renderer=pytorch3d'])

In [3]:
# second, run the code
processor = Image_processor(args_set=args())
inputs = args().inputs
results = processor.run(inputs)

INFO:root:{'tab': 'resnet_cm16_process_images', 'configs_yml': 'configs/image.yml', 'inputs': 'demo/images', 'output_dir': 'demo/image_results2', 'interactive_vis': True, 'show_largest_person_only': False, 'show_mesh_stand_on_image': True, 'soi_camera': 'far', 'make_tracking': False, 'temporal_optimization': False, 'save_dict_results': True, 'save_visualization_on_img': True, 'fps_save': 24, 'character': 'smpl', 'renderer': 'pytorch3d', 'f': None, 'model_return_loss': False, 'model_version': 1, 'multi_person': True, 'new_training': False, 'perspective_proj': False, 'FOV': 60, 'focal_length': 443.4, 'lr_backbone': 0.0003, 'lr': 0.0003, 'adjust_lr_factor': 0.1, 'weight_decay': 1e-06, 'epoch': 50, 'fine_tune': True, 'GPUS': 0, 'batch_size': 64, 'input_size': 512, 'master_batch_size': -1, 'nw': 4, 'optimizer_type': 'Adam', 'pretrain': 'simplebaseline', 'fix_backbone_training_scratch': False, 'dropout': 0.1, 'pre_norm': False, 'backbone_num_channels': 64, 'track_attention': False, 'enc_laye

visualize in gpu mode


INFO:root:start building model.
INFO:root:missing parameters of layers:[]


/z/home/mkhoshle/Human_object_transform/trained_models/pretrain_resnet.pkl
Using HOBJ v1
Confidence: 0.25


INFO:root:using fine_tune model: trained_models/resnet_cm16_V1_epoch_10.pkl
INFO:root:missing parameters of layers:['backbone.0.conv1.weight', 'backbone.0.bn1.weight', 'backbone.0.bn1.bias', 'backbone.0.bn1.running_mean', 'backbone.0.bn1.running_var', 'backbone.0.bn1.num_batches_tracked', 'backbone.0.layer1.0.conv1.weight', 'backbone.0.layer1.0.bn1.weight', 'backbone.0.layer1.0.bn1.bias', 'backbone.0.layer1.0.bn1.running_mean', 'backbone.0.layer1.0.bn1.running_var', 'backbone.0.layer1.0.bn1.num_batches_tracked', 'backbone.0.layer1.0.conv2.weight', 'backbone.0.layer1.0.bn2.weight', 'backbone.0.layer1.0.bn2.bias', 'backbone.0.layer1.0.bn2.running_mean', 'backbone.0.layer1.0.bn2.running_var', 'backbone.0.layer1.0.bn2.num_batches_tracked', 'backbone.0.layer1.0.conv3.weight', 'backbone.0.layer1.0.bn3.weight', 'backbone.0.layer1.0.bn3.bias', 'backbone.0.layer1.0.bn3.running_mean', 'backbone.0.layer1.0.bn3.running_var', 'backbone.0.layer1.0.bn3.num_batches_tracked', 'backbone.0.layer1.0.downs

Initialization finished!
Processing demo/images, saving to demo/image_results2


INFO:root:gathering datasets


Loading 3 images to process
torch.Size([3, 512, 512, 3]) torch.Size([3, 512, 512, 3])
{'mode': 'parsing', 'calc_loss': False}
output


RuntimeError: The size of tensor a (64) must match the size of tensor b (256) at non-singleton dimension 2

In [ ]:
from IPython.display import Image, display
import glob
for img_path in glob.glob('demo/image_results2/*.jpg'):
    display(Image(img_path))

In [ ]:
# optionally, if you want to export the model to ONNX:
# torch.onnx.export(model, x, "faster_rcnn.onnx", opset_version = 11)

In [1]:
from torchvision import models as py_models

In [5]:
from HumanObj_videos_ResNet.lib.models.BackboneWithFPN import _resnet_fpn_extractor

In [6]:
backbone = py_models.resnet50(pretrained=False)

In [7]:
out = _resnet_fpn_extractor(backbone, 5)

TypeError: __init__() got an unexpected keyword argument 'norm_layer'

## Plot Metric 

In [13]:
file_path = "val.txt"
df_val = pd.read_csv(file_path, sep=" ", header=None)
df_val.head()

,0,1,2,3
0,Running,MPJPE:327.5680236816406|0.0;,Running,PAMPJPE:212.51625061035156|0.0
1,Running,MPJPE:306.97698974609375|423.9913330078125;,Running,PAMPJPE:197.36795043945312|229.4709930419922
2,Running,MPJPE:289.9775390625|860.6228637695312;,Running,PAMPJPE:183.23876953125|552.2484741210938
3,Running,MPJPE:268.9407958984375|1973.099609375;,Running,PAMPJPE:172.0282745361328|791.2120971679688
4,Running,MPJPE:253.4237823486328|2541.59228515625;,Running,PAMPJPE:163.92245483398438|895.7868041992188


In [14]:
f = lambda x: x.rstrip(';')
df_val[[1]] = df_val[[1]].applymap(f)
    
f = lambda x: x.replace("MPJPE:", "")
df_val[[1]] = df_val[[1]].applymap(f)

df_val[["MPJPE_mean","MPJPE_var"]] = df_val[1].str.split("|", expand = True)

f = lambda x: float(x)
df_val[["MPJPE_mean","MPJPE_var"]] = df_val[["MPJPE_mean","MPJPE_var"]].applymap(f)

In [15]:
f = lambda x: x.rstrip(';')
df_val[[3]] = df_val[[3]].applymap(f)
    
f = lambda x: x.replace("PAMPJPE:", "")
df_val[[3]] = df_val[[3]].applymap(f)

df_val[["PAMPJPE_mean","PAMPJPE_var"]] = df_val[3].str.split("|", expand = True)

f = lambda x: float(x)
df_val[["PAMPJPE_mean","PAMPJPE_var"]] = df_val[["PAMPJPE_mean","PAMPJPE_var"]].applymap(f)

In [16]:
df_val = df_val.drop([0,1,2,3],axis=1)

In [20]:
df_val['step'] = [i*2000 for i in range(len(df_val))]

In [21]:
df_val.head()

,MPJPE_mean,MPJPE_var,PAMPJPE_mean,PAMPJPE_var,step
0,327.568024,0.000000,212.516251,0.000000,0
1,306.976990,423.991333,197.367950,229.470993,2000
2,289.977539,860.622864,183.238770,552.248474,4000
3,268.940796,1973.099609,172.028275,791.212097,6000
4,253.423782,2541.592285,163.922455,895.786804,8000


In [34]:
from matplotlib.pyplot import *
import matplotlib
import seaborn as sns

plt.rcParams['font.size'] = 9
fig = plt.figure(figsize=(4,4))
axes = fig.add_subplot(1,1,1)

yerrs = [np.sqrt(df_val['MPJPE_var'].values.transpose().tolist()),np.sqrt(df_val['PAMPJPE_var'].values.transpose().tolist())]

df_val.plot(x='step', y=['PAMPJPE_mean','MPJPE_mean'], color=['blue','red'], yerr=yerrs, ax=axes) 

# df_val.loc[:,['PAMPJPE_mean','MPJPE_mean']].plot(color=['blue','red'],yerr=yerrs,ax=axes) 

plt.style.use('ggplot')
sns.set_style('ticks')
axes.set_xlabel('$Iterations$')   
axes.set_ylabel('Performance')

axes.set_ylim(0, 400)
# axes.set_xlim(0, 220)
axes.yaxis.grid(which='major', linewidth='0.5', color='grey')
axes.xaxis.grid(which='major', linewidth='0.5', color='grey')
axes.yaxis.grid(which='minor', linewidth='0.5', color='grey')
axes.xaxis.grid(which='minor', linewidth='0.5', color='grey') 
sns.despine(offset=10, ax=axes)

# lgd = plt.legend(['IO','Compute','Opening Trajectory','Communication'],loc='upper center', bbox_to_anchor=(0.43, -0.38),
                 # frameon=False, shadow=False, ncol=2, prop={'size':12})
plt.tight_layout()
fig.subplots_adjust(bottom=0.4)

fig.savefig('{}.pdf'.format('validation'))